# Challenge: Circuit Extraction

Our normal simplification procedure always yields ZX-diagrams that can be turned back into (ancilla-free) circuits.

**Challenge**: find a means of deterministically realising the ZX-diagram g1 below, _without increasing T-count_.

In [2]:
import random, math, os, time
import sys; sys.path.append('..')
import pyzx as zx

We build a circuit that has 10 T gates, corresponding to phase gadgets of degree 3 or fewer in every possible position before and after a Hadamard gate.

In [44]:
c = zx.qasm(
"""
qreg q[3];
cx q[0], q[2];
cx q[1], q[2];
t q[2];
cx q[0], q[2];
cx q[1], q[2];

cx q[0], q[1];
t q[1];
cx q[0], q[1];

cx q[2], q[1];
t q[1];
cx q[2], q[1];

t q[0];
t q[1];
t q[2];

h q[1];

t q[1];

cx q[0], q[2];
cx q[1], q[2];
t q[2];
cx q[0], q[2];
cx q[1], q[2];

cx q[0], q[1];
t q[1];
cx q[0], q[1];

cx q[2], q[1];
t q[1];
cx q[2], q[1];
""")
zx.draw(c)
g = c.to_graph()
zx.full_reduce(g)
g.normalize()
zx.draw(g)
print(zx.tcount(g))
print(zx.gflow.gflow(g, pauli=True) is not None)

10
True


At this point the graph has gflow, so it will be extractible. However, if we apply more advanced T count reduction procedures like TODD:

In [ ]:
g1 = g.copy()
zx.todd.todd_on_graph(g1)
zx.draw(g1)
print(zx.tcount(g1))

5
None


We can halve the T-count, but the circuit is no longer extractible:

In [34]:
c1 = zx.extract_circuit(g1.copy())

Exception: No extractable vertex found. Something went wrong

...because it no longer has gflow:

In [41]:
print(zx.gflow.gflow(g1, pauli=True))

None


This is evidently still a unitary ZX diagram. Can it be implemented deterministically (e.g. using ancillae and classical control) _without_ increasing the T count?